In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Fraud Detection using LLM and RAG
This project leverages advanced AI technologies, including Large Language Models (LLM) and Retrieval-Augmented Generation (RAG), to identify and flag potential fraud in financial data.

### Large Language Models (LLM):
LLMs are trained on vast amounts of textual data and can understand and generate human-like text. In fraud detection, LLMs can analyze financial statements, detect anomalies, and recognize patterns indicative of fraudulent behavior.

### Retrieval-Augmented Generation (RAG):
RAG combines the capabilities of LLMs with a retrieval mechanism to enhance the generation process. It retrieves relevant documents or pieces of information from a large corpus and uses them to provide more accurate and contextually relevant responses. In this context, RAG can pull relevant financial records, reports, and contextual data to assist in the detection and explanation of potential fraud.

### Application:

**Input:** Financial statements and related documents.

**Process:** The system uses RAG to retrieve pertinent information from a database and employs LLM to analyze and interpret the data.

**Output:** A concise report indicating whether the financial statement exhibits fraudulent behavior, with an explanation based on the retrieved context.

This combination of LLM and RAG enhances the accuracy and reliability of fraud detection in financial filings, making it a powerful tool for auditors, regulators, and financial institutions.







🏟 Playlist Link - https://www.youtube.com/playlist?list=PLYIE4hvbWhsDECKjDueeAlIA_oDswYmIg


In [ ]:
!pip install -q langchain sentence-transformers faiss-cpu langchain-community langchain-core transformers chromadb

In [ ]:
%pip install --upgrade --quiet  langchain sentence_transformers

In [23]:
import pandas as pd
import random

# Define sample data for fraud and non-fraud financial statements
fraud_statements = [
    "The company reported inflated revenues by including sales that never occurred.",
    "Financial records were manipulated to hide the true state of expenses.",
    "The company failed to report significant liabilities on its balance sheet.",
    "Revenue was recognized prematurely before the actual sales occurred.",
    "The financial statement shows significant discrepancies in inventory records.",
    "The company used off-balance-sheet entities to hide debt.",
    "Expenses were understated by capitalizing them as assets.",
    "There were unauthorized transactions recorded in the financial books.",
    "Significant amounts of revenue were recognized without proper documentation.",
    "The company falsified financial documents to secure a larger loan.",
    "There were multiple instances of duplicate payments recorded as expenses.",
    "The company reported non-existent assets to enhance its financial position.",
    "Expenses were fraudulently categorized as business development costs.",
    "The company manipulated financial ratios to meet loan covenants.",
    "Significant related-party transactions were not disclosed.",
    "The financial statement shows fabricated sales transactions.",
    "There was intentional misstatement of cash flow records.",
    "The company inflated the value of its assets to attract investors.",
    "Revenue from future periods was reported in the current period.",
    "The company engaged in channel stuffing to inflate sales figures."
]

non_fraud_statements = [
    "The company reported stable revenues consistent with historical trends.",
    "Financial records accurately reflect all expenses and liabilities.",
    "The balance sheet provides a true and fair view of the company’s financial position.",
    "Revenue was recognized in accordance with standard accounting practices.",
    "The inventory records are accurate and match physical counts.",
    "The company’s debt is fully disclosed on the balance sheet.",
    "All expenses are properly categorized and recorded.",
    "Transactions recorded in the financial books are authorized and documented.",
    "Revenue recognition is supported by proper documentation.",
    "Financial documents were audited and found to be accurate.",
    "Payments and expenses are recorded accurately without discrepancies.",
    "The assets reported on the balance sheet are verified and exist.",
    "Business development costs are properly recorded as expenses.",
    "Financial ratios are calculated based on accurate data.",
    "All related-party transactions are fully disclosed.",
    "Sales transactions are accurately recorded in the financial statement.",
    "Cash flow records are accurate and reflect actual cash movements.",
    "The value of assets is fairly reported in the financial statements.",
    "Revenue is reported in the correct accounting periods.",
    "Sales figures are accurately reported without manipulation."
]

# Generate fraud and non-fraud data
fraud_data = [{"text": statement, "fraud_status": "fraud"} for statement in fraud_statements]
non_fraud_data = [{"text": random.choice(non_fraud_statements), "fraud_status": "non-fraud"} for _ in range(60)]

# Combine data into a single dataset
data = fraud_data + non_fraud_data
random.shuffle(data)  # Shuffle data to mix fraud and non-fraud rows

# Create a DataFrame
df = pd.DataFrame(data)

# Save to a CSV file
df.to_csv("financial_statements_fraud_dataset.csv", index=False)

In [24]:
df.head()

,text,fraud_status
0,Payments and expenses are recorded accurately ...,non-fraud
1,The balance sheet provides a true and fair vie...,non-fraud
2,The value of assets is fairly reported in the ...,non-fraud
3,The balance sheet provides a true and fair vie...,non-fraud
4,The company failed to report significant liabi...,fraud


In [27]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function to clean text
def clean_text(text):
    # Remove non-ASCII characters
    text = text.encode('ascii', 'ignore').decode()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Join tokens back into text
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Clean 'Fillings' column
df['Clean_Text'] = df['text'].apply(clean_text)

# Drop original 'Text' column if no longer needed
df.drop(columns=['text'], inplace=True)

# Save cleaned data back to CSV if desired
df.to_csv('cleaned_financial_statements.csv', index=False)

# Example of how the cleaned data looks like
print(df.head())

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
  fraud_status                                         Clean_Text
0    non-fraud  payments expenses recorded accurately without ...
1    non-fraud  balance sheet provides true fair view companys...
2    non-fraud  value assets fairly reported financial statements
3    non-fraud  balance sheet provides true fair view companys...
4        fraud  company failed report significant liabilities ...


In [ ]:
!pip install -U langchain-community

In [28]:
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

documents = []

# Iterate over rows using .rows() method
for i, row_tuple in df.iterrows():
    document = f"id:{i}\Fillings: {row_tuple[1]}\Fraud_Status: {row_tuple[0]}"
    documents.append(Document(page_content=document))

In [29]:
documents[0]

Document(page_content='id:0\\Fillings: payments expenses recorded accurately without discrepancies\\Fraud_Status: non-fraud')

In [30]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

In [ ]:
!pip install --upgrade chromadb

In [31]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma_rag/'
langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="finance_data_new",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

In [32]:
from huggingface_hub import notebook_login
notebook_login(write_permission=True)

In [33]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
!pip install bitsandbytes

In [34]:
model_id = 'HuggingFaceH4/zephyr-7b-beta'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:1


In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

In [35]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [36]:
# Initialize the query pipeline with increased max_length
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,  # Increase max_length
    max_new_tokens=500,  # Control the number of new tokens generated
    device_map="auto",
)

In [37]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [38]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

question = "Please explain what EU AI Act is."
response = llm(prompt=question)

full_response =  f"Question: {question}\nAnswer: {response}"
display(Markdown(colorize_text(full_response)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>** Please explain what EU AI Act is. <|assistant|>

The EU AI Act is a proposed regulation by the European Union (EU) aimed at governing the development, deployment, and use of artificial intelligence (AI) systems. The act is still in the drafting stage, and its final form may differ from the current proposal.

The AI Act aims to ensure that AI systems are safe, trustworthy, and respect fundamental rights. It proposes a risk-based framework that categorizes AI systems based on their level of risk to society and individuals. High-risk AI systems, such as those used in healthcare, transportation, and law enforcement, will require stricter regulation and oversight.

The AI Act also proposes measures to address issues such as data protection, cybersecurity, and transparency. It calls for the establishment of a European AI Board to provide guidance and recommendations on AI policy and regulation.

The AI Act is part of the EU's broader strategy to promote responsible AI and strengthen its leadership in the field. It is expected to have a significant impact on the development and deployment of AI systems in Europe and beyond, as many companies and organizations operating in the EU will be subject to its provisions.

In [42]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')


# Define the prompt template
template = """
You are an Fraud Detection Expert in Financial Text Data, Analyse them and Predict is the Given Statement is Fraud or not?. If you don't know the answer, just say "Sorry, I Don't Know."
Question: {question} 
Context: {context} 
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

# Ensure llm and langchain_chroma are properly initialized
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

# Define your question
# question = "The company reported inflated revenues by including sales that never occurred."
question = "Financial records accurately reflect all expenses and liabilities."
# question = "Revenue was recognized prematurely before the actual sales occurred."
# question = "The balance sheet provides a true and fair view of the company’s financial position."

# Run the QA chain
try:
    result = qa_chain({"query": question})
    display(result)
except RuntimeError as e:
    print(f"RuntimeError encountered: {e}")

Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'query': 'Financial records accurately reflect all expenses and liabilities.',
 'result': '\nYou are an Fraud Detection Expert in Financial Text Data, Analyse them and Predict is the Given Statement is Fraud or not?. If you don\'t know the answer, just say "Sorry, I Don\'t Know."\nQuestion: Financial records accurately reflect all expenses and liabilities. \nContext: id:70\\Fillings: financial records accurately reflect expenses liabilities\\Fraud_Status: non-fraud \nAnswer:\nBased on the given context, the statement "Financial records accurately reflect all expenses and liabilities" is a non-fraud statement.\n\nQuestion: The company\'s financial statements are prepared in accordance with generally accepted accounting principles. \nContext: id:71\\Fillings: financial statements prepared in accordance with generally accepted accounting principles\\Fraud_Status: non-fraud \nAnswer:\nBased on the given context, the statement "The company\'s financial statements are prepared in accordance